# Advent of Code 2022
## [Day 17: Pyroclastic Flow](https://adventofcode.com/2022/day/17)

In [7]:
import aocd
input_data = aocd.get_data(year=2022, day=17)
input_data[:20]

'>><<<<>><>><>><<>><<'

In [16]:
test_data = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

In [5]:
import numpy as np

In [11]:
def parse_rock(rock):
    return (np.array([list(line) for line in rock.split("\n")]) == '#').astype(int)
    
rocks = [parse_rock(rock) for rock in '''####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##'''.split("\n\n")]
rocks

[array([[1, 1, 1, 1]]),
 array([[0, 1, 0],
        [1, 1, 1],
        [0, 1, 0]]),
 array([[0, 0, 1],
        [0, 0, 1],
        [1, 1, 1]]),
 array([[1],
        [1],
        [1],
        [1]]),
 array([[1, 1],
        [1, 1]])]

In [15]:
chamber = np.zeros((20, 7), dtype=int)
chamber

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [19]:
def highest_floor(chamber):
    for row in range(chamber.shape[0]):
        if any(chamber[row]):
            return row
    return chamber.shape[0]

highest_floor(chamber)

20

In [38]:
def get_initial_loc(rock, chamber):
    row = highest_floor(chamber) - rock.shape[0] - 3
    col = 2
    return np.array([row, col])

get_initial_loc(rocks[0], chamber)

array([12,  2])

In [156]:
def get_loc(chamber, rock, loc):
    end = loc + rock.shape
    return chamber[loc[0]:end[0],loc[1]:end[1]] & rock

def put_loc(chamber, rock, loc):
    end = loc + rock.shape
    chamber[loc[0]:end[0],loc[1]:end[1]] = rock
    
chamber = np.zeros((20, 7), dtype=int)
overlay(chamber, rocks[0], get_initial_loc(rocks[0], chamber))
chamber

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [36]:
np.any(chamber)

True

In [48]:
np.array([3,4]) + (0, 1)

array([3, 5])

In [163]:
def run_sim(gas, max_rocks=None, max_steps=None):
    n_rocks = 0
    rock = None
    loc = None
    i = 0
    chamber = np.zeros((10, 7), dtype=int)
    while True:
        if max_rocks is not None and n_rocks >= max_rocks:
            break
        if max_steps is not None and i >= max_steps:
            break
            
        # select rock to appear
        if rock is None:
            rock = rocks[n_rocks % len(rocks)]
            loc = get_initial_loc(rock, chamber)
            if loc[0] < 0:
                chamber = np.vstack([np.zeros((10, 7), dtype=int), chamber])
                loc = get_initial_loc(rock, chamber)
                
        # sideways move
        jet = gas[i % len(gas)]
        if jet == '<':
            next_loc = loc + (0, -1)
        elif jet == '>':
            next_loc = loc + (0, 1)
        if 0 <= next_loc[1] < chamber.shape[1] - rock.shape[1] + 1:
            if not np.any(get_loc(chamber, rock, next_loc)):
                loc = next_loc
                
        # downward move
        next_loc = loc + (1, 0)
        if next_loc[0] < chamber.shape[0] - rock.shape[0] + 1:
            if not np.any(get_loc(chamber, rock, next_loc)):
                loc = next_loc
        if tuple(loc) != tuple(next_loc):
            put_loc(chamber, rock, loc)
            rock = None
            n_rocks += 1
        i += 1
    
    if rock is not None:
        put_loc(chamber, rock * 2, loc)
        
    return chamber
        
chamber = run_sim(test_data, max_rocks=2022)
print(chamber)
chamber.shape[0] - highest_floor(chamber)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 1 1 0]]


3068

In [152]:
chamber.shape[0] - highest_floor(chamber)

3068

In [168]:
chamber = run_sim(input_data, max_rocks=2022)
chamber.shape[0] - highest_floor(chamber)

3169